In [1]:
import os
import sys

module_path = os.path.abspath(os.path.join('..', '..', 'news-bias-agent'))
if module_path not in sys.path:
    sys.path.append(module_path)

%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
from src.dataLoader import NewsDataLoader
from src.biasAnalyzer import BiasAnalyzer
from src.prompts import BIAS_ANALYSIS_SYSTEM_PROMPT
import time
from tqdm import tqdm
import asyncio

In [3]:
loader = NewsDataLoader(data_path=os.path.join('..','data', 'News_Category_Dataset_v3.json'))
analyzer = BiasAnalyzer(system_prompt=BIAS_ANALYSIS_SYSTEM_PROMPT, temperature=0.1)

In [8]:
test_df = loader.get_high_bias_categories(n=5)
articles = loader.to_dict_list(test_df)
articles

[{'headline': 'An Underdog In Michigan’s Governor Race Shows Dems How To Run On Climate Change',
  'short_description': 'Abdul El-Sayed, the 33-year-old former Detroit health commissioner, has a plan to fight environmental racism.',
  'category': 'POLITICS'},
 {'headline': "Teachers Union Claims The 'Trump Effect' Is Warping Kids' Minds",
  'short_description': "The National Education Association says Donald Trump's anti-immigrant stance is emboldening schoolyard bullies.",
  'category': 'POLITICS'},
 {'headline': '4 Tips for Female Executives As They Climb the Corporate Ladder',
  'short_description': 'Women frequently do not have the same level of support or development opportunities that men enjoy -- because they lack peers and mentors. Here are some tips I share with the female executives I coach, as they climb the corporate ladder.',
  'category': 'BUSINESS'},
 {'headline': 'Melania Trump Is Forging Her Own Path As First Lady, In Spite Of Her Husband',
  'short_description': 'Thou

In [9]:
result = analyzer.analyze_news(
    headline=articles[0]['headline'],
    description=articles[0]['short_description']
)
result

BiasAnalysisOutput(sensationalism_score=4, emotional_charge=3, political_leaning='Left', sentiment='Positive', bias_indicators=['Underdog', 'Shows Dems How To Run On Climate Change', 'fight environmental racism'], framing='The story frames Abdul El-Sayed as an innovative and effective model for Democrats on climate change and environmental justice, highlighting his specific plan.', subjectivity_flag=True, reasoning='The headline uses subjective language like "Underdog" and "Shows Dems How To Run," indicating a positive framing of the candidate and his approach. It explicitly aligns with Democratic political strategies and focuses on issues like climate change and environmental racism, which are typically left-leaning concerns.')

AttributeError: 'str' object has no attribute 'pretty_print'